In [1]:
from kmeans.numpy_kmeans import KMeans
from preprocess import load_model
from configs import *
import torch
import numpy as np

In [2]:
corpus, dictionary, tfidf, lsi, mSimilar = load_model("./preprocess/model/")

2021-04-09 15:39:52,744 : INFO : loaded corpus index from ./preprocess/model/corpus.mm.index
2021-04-09 15:39:52,744 : INFO : initializing cython corpus reader from ./preprocess/model/corpus.mm
2021-04-09 15:39:52,759 : INFO : accepted corpus with 9804 documents, 255478 features, 6928768 non-zero entries
2021-04-09 15:39:52,759 : INFO : loading Dictionary object from ./preprocess/model/model.dic
2021-04-09 15:39:53,148 : INFO : Dictionary lifecycle event {'fname': './preprocess/model/model.dic', 'datetime': '2021-04-09T15:39:53.132082', 'gensim': '4.0.1', 'python': '3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}
2021-04-09 15:39:53,148 : INFO : loading TfidfModel object from ./preprocess/model/model.tfidf
2021-04-09 15:39:53,580 : INFO : TfidfModel lifecycle event {'fname': './preprocess/model/model.tfidf', 'datetime': '2021-04-09T15:39:53.580897', 'gensim': '4.0.1', 'python': '3.9.2 (tags/v3.

In [5]:
X = []
for vec_bow in corpus:
    vec_tfidf = tfidf[vec_bow]
    # pprint(vec_tfidf)
    # 转成lsi向量
    vec_lsi = lsi[vec_tfidf]
    X.append([i[1] for i in vec_lsi])

In [4]:
# import torch.nn.functional as F
X = torch.from_numpy(np.array(X))
# F.normalize(X,p=2,dim=0)

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [6]:
from file_utils import get_files

f_list = get_files("./corpus/train/")
l = np.array([int(i.split("\\")[-1].split("-")[0][1:]) for i in f_list])

In [9]:
max_acc = 0
max_i = -1
from training_utils import best_map

#14
for i in range(1000):
    kmeans = KMeans(20, random_state=i, device=device)
    label, _ = kmeans.fit_predict(X)
    acc, _ = best_map(l, label.numpy())
    if acc > max_acc:
        max_acc = acc
        max_i = i
    if acc > 0.45:
        print(i, acc)
print("max", max_i, max_acc)

KeyboardInterrupt: 

In [7]:
from training_utils import best_map
kmeans = KMeans(20, random_state=14, device=device)
label, _ = kmeans.fit_predict(X)
acc, _ = best_map(l, label.numpy())
print(acc)


ValueError: too many values to unpack (expected 2)